# LLM Optimization Modelling Experiment

In [4]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
from IPython.display import Markdown

## 1. Define the problem description

In [39]:
problem = '''Imagine you own a small shop, and you're trying to manage your employees' working hours and sales targets efficiently.
You have three kinds of goals overall: sales goals, employee hours goals and overtime goals.

Sales Goals: 
You have a target of selling 5500 units. 
But sometimes, you might sell fewer (negative deviation) or more (positive deviation) than this target. 
On average, your full-time employees will sell 5 records per hour and your part-time employees will sell 2 records per hour.

Employee Hours: 
You have both full-time and part-time employees. 
Your full-time employees are expected to work a total of 800 hours, while part-time employees are expected to work 320 hours. 
Similarly, like sales, there could be deviations in these working hours. 

Overtime: 
Additionally, your employees might end up working overtime. 
You have a target of 100 hours of overtime for full-time employees.
You are not concerned about limiting the overtime of part-time employees.
Again, you might deviate from this goal.

Each of these deviations has a different priority for you. 
P1 to P4 are priority factors for each deviation.
Your most important goal is to achieve your sales goal.
Second to that is the goal to not exceed 100 overtime hours for full-time employees.
Then, you next priority is to fully utilize your full-time and part time employees.
Your last goal with the lowest priority, you want to keep general overtime hours for both kinds of employees as low as possible.
In addition to the general priorities of you goals, you are twice as concerned about fully utilizing the full-time employee hours.
Finally, overtime hours of part-time employees have a relative cost of three times that of full-time employees.

Now, the goal here is to minimize the overall impact of these deviations, considering their priorities. 
This means finding the best balance between sales targets, employee hours, and overtime, while keeping in mind which deviations are more critical for your business.'''

## 2. Generate the mathematical model

In [51]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer.
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [52]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Mathematical Optimization Model for Employee Management

**Parameters:**

* **S_target:** Target sales (units) = 5500
* **S_FTH:** Sales per hour for full-time employees (units/hour) = 5
* **S_PTH:** Sales per hour for part-time employees (units/hour) = 2
* **H_FT:** Target hours for full-time employees (hours) = 800
* **H_PT:** Target hours for part-time employees (hours) = 320
* **OT_FT_target:** Target overtime hours for full-time employees (hours) = 100
* **P1:** Priority factor for sales deviation
* **P2:** Priority factor for full-time overtime deviation
* **P3:** Priority factor for full-time employee hours deviation
* **P4:** Priority factor for part-time employee hours deviation
* **P5:** Priority factor for overall overtime deviation

**Decision Variables:**

* **S_dev:** Deviation from sales target (units)
* **H_FT_dev:** Deviation from full-time employee hours target (hours)
* **H_PT_dev:** Deviation from part-time employee hours target (hours)
* **OT_FT_dev:** Deviation from full-time overtime target (hours)
* **OT_FT:** Total overtime hours for full-time employees (hours)
* **OT_PT:** Total overtime hours for part-time employees (hours)

**Objective Function:**

Minimize: 
  **P1 * |S_dev| + P2 * |OT_FT_dev| + 2*P3 * |H_FT_dev| + P4 * |H_PT_dev| + P5 * (|OT_FT| + 3*|OT_PT|)**

**Constraints:**

* **Sales:** S_target + S_dev = S_FTH * (H_FT + H_FT_dev + OT_FT) + S_PTH * (H_PT + H_PT_dev + OT_PT) 
* **Full-time overtime:** OT_FT = OT_FT_target + OT_FT_dev
* **Non-negativity:** OT_FT ≥ 0, OT_PT ≥ 0 

**Explanation:**

* **Objective Function:** This function calculates the weighted sum of deviations from each goal. The priority factors (P1-P5) determine the relative importance of each deviation. The higher the priority factor, the more important it is to minimize that deviation. Note that we use the absolute values of the deviations to penalize both positive and negative deviations. Additionally, we incorporate the doubled concern for full-time employee hours utilization and the tripled cost of part-time overtime.
* **Sales Constraint:** This constraint ensures that the total sales, including deviations, are equal to the sum of sales generated by full-time and part-time employees, taking into account their regular hours, deviations from regular hours, and overtime hours.
* **Full-time overtime constraint:** This constraint connects the full-time overtime deviation variable with the total full-time overtime and its target.
* **Non-negativity constraints:** These constraints ensure that overtime hours for both employee types are non-negative.

**Solving the Model:**

This model can be solved using optimization software (e.g., Excel Solver, Python libraries like PuLP or OR-Tools) to determine the optimal values for the decision variables that minimize the objective function while satisfying the constraints. The solution will provide the best balance between sales targets, employee hours, and overtime, considering the given priorities.


## 3. Generate the pyomo code

In [53]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt2 += response.text 
response2 = generative_multimodal_model.generate_content([prompt2])

In [54]:
#Showing the code in a formatted way
Markdown(response2.text)

```python
import pyomo.environ as pyo

# Sample data (replace with actual data if available)
S_target = 5500  # Target sales (units)
S_FTH = 5  # Sales per hour for full-time employees (units/hour)
S_PTH = 2  # Sales per hour for part-time employees (units/hour)
H_FT = 800  # Target hours for full-time employees (hours)
H_PT = 320  # Target hours for part-time employees (hours)
OT_FT_target = 100  # Target overtime hours for full-time employees (hours)
P1 = 1  # Priority factor for sales deviation
P2 = 1.2  # Priority factor for full-time overtime deviation
P3 = 1.5  # Priority factor for full-time employee hours deviation
P4 = 0.8  # Priority factor for part-time employee hours deviation
P5 = 2  # Priority factor for overall overtime deviation

# Create a concrete model
model = pyo.ConcreteModel()

# Decision variables
model.S_dev = pyo.Var(domain=pyo.Reals)  # Deviation from sales target
model.H_FT_dev = pyo.Var(domain=pyo.Reals)  # Deviation from full-time employee hours target
model.H_PT_dev = pyo.Var(domain=pyo.Reals)  # Deviation from part-time employee hours target
model.OT_FT_dev = pyo.Var(domain=pyo.Reals)  # Deviation from full-time overtime target
model.OT_FT = pyo.Var(domain=pyo.NonNegativeReals)  # Total overtime hours for full-time employees
model.OT_PT = pyo.Var(domain=pyo.NonNegativeReals)  # Total overtime hours for part-time employees

# Objective function
def objective_rule(model):
    return P1 * abs(model.S_dev) + P2 * abs(model.OT_FT_dev) + 2*P3 * abs(model.H_FT_dev) + P4 * abs(model.H_PT_dev) + P5 * (abs(model.OT_FT) + 3*abs(model.OT_PT))
model.objective = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# Constraints
model.sales_constraint = pyo.Constraint(expr=S_target + model.S_dev == S_FTH * (H_FT + model.H_FT_dev + model.OT_FT) + S_PTH * (H_PT + model.H_PT_dev + model.OT_PT))
model.overtime_constraint = pyo.Constraint(expr=model.OT_FT == OT_FT_target + model.OT_FT_dev)

# Solve the model
solver = pyo.SolverFactory('glpk')  # You can use other solvers like 'gurobi' or 'cplex' if available
solver.solve(model)

# Print the results
print("Optimal Solution:")
print("Sales Deviation:", model.S_dev.value)
print("Full-Time Hours Deviation:", model.H_FT_dev.value)
print("Part-Time Hours Deviation:", model.H_PT_dev.value)
print("Full-Time Overtime Deviation:", model.OT_FT_dev.value)
print("Total Full-Time Overtime:", model.OT_FT.value)
print("Total Part-Time Overtime:", model.OT_PT.value)
print("Total Cost:", model.objective.expr())
```

This code defines the Pyomo model for the employee management problem. It sets up the variables, objective function, and constraints as defined in the mathematical model. 

**Note:** This code uses sample data for parameters. Remember to replace this sample data with your actual data when implementing the model. 


## 4. Input problem data and try running the generated code

In [56]:
import pyomo.environ as pyo

# Sample data (replace with actual data if available)
S_target = 5500  # Target sales (units)
S_FTH = 5  # Sales per hour for full-time employees (units/hour)
S_PTH = 2  # Sales per hour for part-time employees (units/hour)
H_FT = 800  # Target hours for full-time employees (hours)
H_PT = 320  # Target hours for part-time employees (hours)
OT_FT_target = 100  # Target overtime hours for full-time employees (hours)
P1 = 1  # Priority factor for sales deviation
P2 = 1.2  # Priority factor for full-time overtime deviation
P3 = 1.5  # Priority factor for full-time employee hours deviation
P4 = 0.8  # Priority factor for part-time employee hours deviation
P5 = 2  # Priority factor for overall overtime deviation

# Create a concrete model
model = pyo.ConcreteModel()

# Decision variables
model.S_dev = pyo.Var(domain=pyo.Reals)  # Deviation from sales target
model.H_FT_dev = pyo.Var(domain=pyo.Reals)  # Deviation from full-time employee hours target
model.H_PT_dev = pyo.Var(domain=pyo.Reals)  # Deviation from part-time employee hours target
model.OT_FT_dev = pyo.Var(domain=pyo.Reals)  # Deviation from full-time overtime target
model.OT_FT = pyo.Var(domain=pyo.NonNegativeReals)  # Total overtime hours for full-time employees
model.OT_PT = pyo.Var(domain=pyo.NonNegativeReals)  # Total overtime hours for part-time employees

# Objective function
def objective_rule(model):
    return P1 * abs(model.S_dev) + P2 * abs(model.OT_FT_dev) + 2*P3 * abs(model.H_FT_dev) + P4 * abs(model.H_PT_dev) + P5 * (abs(model.OT_FT) + 3*abs(model.OT_PT))
model.objective = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# Constraints
model.sales_constraint = pyo.Constraint(expr=S_target + model.S_dev == S_FTH * (H_FT + model.H_FT_dev + model.OT_FT) + S_PTH * (H_PT + model.H_PT_dev + model.OT_PT))
model.overtime_constraint = pyo.Constraint(expr=model.OT_FT == OT_FT_target + model.OT_FT_dev)

# Solve the model
solver = pyo.SolverFactory('ipopt')  # You can use other solvers like 'gurobi' or 'cplex' if available
solver.solve(model)

# Print the results
print("Optimal Solution:")
print("Sales Deviation:", model.S_dev.value)
print("Full-Time Hours Deviation:", model.H_FT_dev.value)
print("Part-Time Hours Deviation:", model.H_PT_dev.value)
print("Full-Time Overtime Deviation:", model.OT_FT_dev.value)
print("Total Full-Time Overtime:", model.OT_FT.value)
print("Total Part-Time Overtime:", model.OT_PT.value)
print("Total Cost:", model.objective.expr())

ValueError: Cannot load a SolverResults object with bad status: error

## 5. Correct the code to verify model viability (optional)

## 6. Printing the outputs as strings, so they can be saved.
Those can be rendered as markdown for better readability

In [59]:
print(response.text)

## Mathematical Optimization Model for Employee Management

**Parameters:**

* **S_target:** Target sales (units) = 5500
* **S_FTH:** Sales per hour for full-time employees (units/hour) = 5
* **S_PTH:** Sales per hour for part-time employees (units/hour) = 2
* **H_FT:** Target hours for full-time employees (hours) = 800
* **H_PT:** Target hours for part-time employees (hours) = 320
* **OT_FT_target:** Target overtime hours for full-time employees (hours) = 100
* **P1:** Priority factor for sales deviation
* **P2:** Priority factor for full-time overtime deviation
* **P3:** Priority factor for full-time employee hours deviation
* **P4:** Priority factor for part-time employee hours deviation
* **P5:** Priority factor for overall overtime deviation

**Decision Variables:**

* **S_dev:** Deviation from sales target (units)
* **H_FT_dev:** Deviation from full-time employee hours target (hours)
* **H_PT_dev:** Deviation from part-time employee hours target (hours)
* **OT_FT_dev:** Deviation

In [60]:
print(response2.text)

```python
import pyomo.environ as pyo

# Sample data (replace with actual data if available)
S_target = 5500  # Target sales (units)
S_FTH = 5  # Sales per hour for full-time employees (units/hour)
S_PTH = 2  # Sales per hour for part-time employees (units/hour)
H_FT = 800  # Target hours for full-time employees (hours)
H_PT = 320  # Target hours for part-time employees (hours)
OT_FT_target = 100  # Target overtime hours for full-time employees (hours)
P1 = 1  # Priority factor for sales deviation
P2 = 1.2  # Priority factor for full-time overtime deviation
P3 = 1.5  # Priority factor for full-time employee hours deviation
P4 = 0.8  # Priority factor for part-time employee hours deviation
P5 = 2  # Priority factor for overall overtime deviation

# Create a concrete model
model = pyo.ConcreteModel()

# Decision variables
model.S_dev = pyo.Var(domain=pyo.Reals)  # Deviation from sales target
model.H_FT_dev = pyo.Var(domain=pyo.Reals)  # Deviation from full-time employee hours target
mode